In [79]:
# path to our tables in the silver layer
silver_tables_path = "abfss://3201caea-2820-458c-8915-9fe2f28ba2a7@onelake.dfs.fabric.microsoft.com/4e18f678-60fe-4988-82c1-2d51453db35e/Tables"

StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 81, Finished, Available)

In [80]:
df_sales = spark.read.option("delimiter", "\t").format("csv").option("header", "True").load("Files/Bronze_AdventureWorks/Sales_19_04_2024.csv")

StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 82, Finished, Available)

## specifying datatypes

In [81]:
from pyspark.sql.functions import col, regexp_replace, when, substring, to_date, date_format, regexp_extract
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

# df_sales = df_sales.withColumn("SalesOrderNumber", 
#                                    when(col("SalesOrderNumber").isNull(), None)
#                                    .otherwise(regexp_replace(col("SalesOrderNumber").cast("string"), '[^0-9.]+', '')).cast("int"))
df_sales = df_sales.withColumn("OrderDate", to_date(col("OrderDate"), "EEEE, MMMM dd, yyyy"))
df_sales = df_sales.withColumn("ProductKey", col("ProductKey").cast("int"))
df_sales = df_sales.withColumn("ResellerKey", col("ResellerKey").cast("int"))
df_sales = df_sales.withColumn("EmployeeKey", col("EmployeeKey").cast("int"))
df_sales = df_sales.withColumn("SalesTerritoryKey", col("SalesTerritoryKey").cast("int"))
df_sales = df_sales.withColumn("Quantity", col("Quantity").cast("int"))
df_sales = df_sales.withColumn("Unit Price", 
                                   when(col("Unit Price").isNull(), None)
                                   .otherwise(regexp_replace(col("Unit Price").cast("string"), '[^0-9.]+', '')))
df_sales = df_sales.withColumn("Unit Price", col("Unit Price").cast("float"))
df_sales = df_sales.withColumn("Sales", 
                                   when(col("Sales").isNull(), None)
                                   .otherwise(regexp_replace(col("Sales").cast("string"), '[^0-9.]+', '')))
df_sales = df_sales.withColumn("Sales", col("Sales").cast("float"))
df_sales = df_sales.withColumn("Cost", 
                                   when(col("Cost").isNull(), None)
                                   .otherwise(regexp_replace(col("Cost").cast("string"), '[^0-9.]+', '')))
df_sales = df_sales.withColumn("Cost", col("Cost").cast("float"))

StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 83, Finished, Available)

## Changing the column names

In [82]:
df_sales = df_sales.withColumnRenamed("Unit Price", "UnitPrice")

StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 84, Finished, Available)

## slowly changing dimensions

In [83]:
# adding columns for SCD
from pyspark.sql.functions import lit, current_date

df_sales = df_sales.withColumn("isActive", lit(True))
df_sales = df_sales.withColumn("startDate", date_format(current_date(), "yyyy-MM-dd").cast("date"))
df_sales = df_sales.withColumn("endDate", lit("9999-12-31").cast("date"))

StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 85, Finished, Available)

## Loading the new data

In [84]:
df_sales_new = spark.read.option("delimiter", "\t").format("csv").option("header", "True").load("Files/Bronze_AdventureWorks/Sales_new_29_04_2024.csv")
# Datatypes
# df_sales_new = df_sales_new.withColumn("SalesOrderNumber", 
#                                    when(col("SalesOrderNumber").isNull(), None)
#                                    .otherwise(regexp_replace(col("SalesOrderNumber").cast("string"), '[^0-9.]+', '')).cast("int"))

df_sales_new = df_sales_new.withColumn("OrderDate", to_date(col("OrderDate"), "EEEE, MMMM dd, yyyy"))
df_sales_new = df_sales_new.withColumn("ProductKey", col("ProductKey").cast("int"))
df_sales_new = df_sales_new.withColumn("ResellerKey", col("ResellerKey").cast("int"))
df_sales_new = df_sales_new.withColumn("EmployeeKey", col("EmployeeKey").cast("int"))
df_sales_new = df_sales_new.withColumn("SalesTerritoryKey", col("SalesTerritoryKey").cast("int"))
df_sales_new = df_sales_new.withColumn("Quantity", col("Quantity").cast("int"))
df_sales_new = df_sales_new.withColumn("Unit Price", 
                                   when(col("Unit Price").isNull(), None)
                                   .otherwise(regexp_replace(col("Unit Price").cast("string"), '[^0-9.]+', '')))
df_sales_new = df_sales_new.withColumn("Unit Price", col("Unit Price").cast("float"))
df_sales_new = df_sales_new.withColumn("Sales", 
                                   when(col("Sales").isNull(), None)
                                   .otherwise(regexp_replace(col("Sales").cast("string"), '[^0-9.]+', '')))
df_sales_new = df_sales_new.withColumn("Sales", col("Sales").cast("float"))
df_sales_new = df_sales_new.withColumn("Cost", 
                                   when(col("Cost").isNull(), None)
                                   .otherwise(regexp_replace(col("Cost").cast("string"), '[^0-9.]+', '')))
df_sales_new = df_sales_new.withColumn("Cost", col("Cost").cast("float"))
# Column names
df_sales_new = df_sales_new.withColumnRenamed("Unit Price", "UnitPrice")

# columns added for scd
df_sales_new = df_sales_new.withColumn("isActive", lit(True))
df_sales_new = df_sales_new.withColumn("startDate", date_format(current_date(), "yyyy-MM-dd").cast("date"))
df_sales_new = df_sales_new.withColumn("endDate", lit("9999-12-31").cast("date"))

display(df_sales_new)


StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 86, Finished, Available)

SynapseWidget(Synapse.DataFrame, f9d84044-4ddd-4f7d-9c1e-1d23b7738cad)

## writing and loading silver

In [85]:
from pyspark.sql.functions import sha2, concat_ws

# columns_except_employee_key = [col_name for col_name in df_sales.columns if col_name != "SalesOrderNumber" and col_name != "startDate" and col_name != "endDate"]
columns_except_employee_key = [col_name for col_name in df_sales.columns if col_name != "startDate" and col_name != "endDate"]
concatenated_columns = concat_ws("||", *columns_except_employee_key)
df_sales = df_sales.withColumn("hashdiff", sha2(concatenated_columns, 256))
df_sales_new = df_sales_new.withColumn("hashdiff", sha2(concatenated_columns, 256))

# Creation of the mergekey
df_sales_new = df_sales_new.withColumn("MergeKey", df_sales_new["EmployeeKey"])

display(df_sales)
display(df_sales_new)


StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 87, Finished, Available)

SynapseWidget(Synapse.DataFrame, f9a576d8-82a2-4595-a777-00337411298e)

SynapseWidget(Synapse.DataFrame, b2088aad-0435-4261-8191-271149df92ad)

## Check if the silver table already exists, otherwise initialize it

In [86]:
try:
    df_sales_silver = spark.read.format("delta").load(f"{silver_tables_path}/Sales")
except Exception as e:
    print("error:", e)
    df_sales.write.format("delta").mode("overwrite").save(f"{silver_tables_path}/Sales")

df_sales_silver = spark.read.format("delta").load(f"{silver_tables_path}/Sales")


StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 88, Finished, Available)

error: [PATH_NOT_FOUND] Path does not exist: abfss://3201caea-2820-458c-8915-9fe2f28ba2a7@onelake.dfs.fabric.microsoft.com/4e18f678-60fe-4988-82c1-2d51453db35e/Tables/Sales.


## Rename columns of df_sales_silver

In [87]:
df_sales_silver = df_sales_silver.select([col(col_name).alias("silver_" + col_name) for col_name in df_sales_silver.columns])

StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 89, Finished, Available)

## SCD Step 1: Left join to get the upserts

In [88]:
from pyspark.sql.functions import expr

df_sales_joined = df_sales_new.join(
    df_sales_silver,
    (df_sales_new["SalesOrderNumber"] == df_sales_silver["silver_SalesOrderNumber"]) &
    (df_sales_new["ProductKey"] == df_sales_silver["silver_ProductKey"]),
    "left"
) \
.where(expr("silver_isActive = True AND silver_hashdiff != hashdiff"))


# With a left anti-join we figure out out what the upserts 
# https://spark.apache.org/docs/latest/sql-ref-syntax-qry-select-join.html#:~:text=An%20anti%20join%20returns%20values,as%20a%20left%20anti%20join.
df_sales_insert = df_sales_new.join(
    df_sales_silver,
    df_sales_new['SalesOrderNumber'] == df_sales_silver['silver_SalesOrderNumber'],
    'left_anti'
)

# We set the mergekeys to none/key in order for the merge operation to recognize if it's an insert or an update
df_sales_joined = df_sales_joined.withColumn("MergeKey", lit(None))
df_sales_insert = df_sales_insert.withColumn("MergeKey", lit(None))
# This df is a copy of the joined df, but with the key set to SalesOrderNumber to make the update work
df_sales_upsert = df_sales_joined.withColumn("MergeKey", df_sales_joined["SalesOrderNumber"])

# display(df_sales_upsert)
display(df_sales_insert)
display(df_sales_joined)

StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 90, Finished, Available)

SynapseWidget(Synapse.DataFrame, b69692da-e87c-497e-9029-72d03147c1e2)

SynapseWidget(Synapse.DataFrame, afa64961-9505-453e-b121-2d1165da235b)

In [89]:
from pyspark.sql.functions import col, lit, when, concat_ws, sha2
from pyspark.sql import SparkSession

# Drop columns from silver after the join, assuming df_sales_joined has the joined data
silver_columns = [col_name for col_name in df_sales_joined.columns if col_name.startswith("silver_")]
df_sales_joined = df_sales_joined.drop(*silver_columns)
df_sales_upsert = df_sales_upsert.drop(*silver_columns)

# We make a union of all the rows that need to be inserted or updated
df_sales_joined = df_sales_joined.union(df_sales_upsert)
df_sales_joined = df_sales_joined.union(df_sales_insert)

display(df_sales_joined)

StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 91, Finished, Available)

SynapseWidget(Synapse.DataFrame, ebb36fac-951b-408a-aa6e-3caae9772452)

In [90]:
from delta.tables import *
from pyspark.sql import functions as F

silver_delta_table = DeltaTable.forPath(spark, f"{silver_tables_path}/Sales")
df_sales_silver = spark.read.format("delta").load(f"{silver_tables_path}/Sales")

display(df_sales_joined)

# Merge for inserting new records
silver_delta_table.alias('silver') \
    .merge(
        df_sales_joined.alias('insert'),
        F.expr('silver.SalesOrderNumber = insert.MergeKey') & F.expr('silver.ProductKey = insert.ProductKey')
    ) \
    .whenNotMatchedInsert(values={
        "SalesOrderNumber": "insert.SalesOrderNumber",
        "OrderDate": "insert.OrderDate",
        "ProductKey": "insert.ProductKey",
        "ResellerKey": "insert.ResellerKey",
        "EmployeeKey": "insert.EmployeeKey",
        "SalesTerritoryKey": "insert.SalesTerritoryKey",
        "Quantity": "insert.Quantity",
        "UnitPrice": "insert.UnitPrice",
        "Sales": "insert.Sales",
        "Cost": "insert.Cost",
        "isActive": "insert.isActive",
        "startDate": "insert.startDate",
        "endDate": "insert.endDate",
        "hashdiff": "insert.hashdiff"
    }) \
    .whenMatchedUpdate(
        condition=F.expr("silver.hashdiff != insert.hashdiff"),
        set={
            "isActive": F.lit(False),
            "endDate": F.current_date()
        }
    ) \
    .execute()

# Reload the Delta table to verify the changes
df_sales_silver = spark.read.format("delta").load(f"{silver_tables_path}/Sales")


condition = (
    (df_sales_silver["SalesOrderNumber"] == "SO43897")
)

display(df_sales_silver.where(condition))

StatementMeta(, 06115ff3-fb7b-4438-9243-2e7a1f466649, 92, Finished, Available)

SynapseWidget(Synapse.DataFrame, 846e1d24-d074-444e-b95d-42d6985e6760)

SynapseWidget(Synapse.DataFrame, 269f1153-aa32-4a66-b52c-b40ce70ff3c7)

In [107]:
df_sales = spark.read.option("delimiter", "\t").format("csv").option("header", "True").load("Files/Bronze_AdventureWorks/Sales_19_04_2024.csv")

from pyspark.sql.functions import when, col
from pyspark.sql import Row


df_sales = df_sales.withColumn("Quantity", when((df_sales["SalesOrderNumber"] == "SO43897") & (df_sales["ProductKey"] == 235), 123).otherwise(df_sales["Quantity"]))


condition = (
    (df_sales["SalesOrderNumber"] == "SO43897") & (df_sales["ProductKey"] == 235)
)

df_sales_changed = df_sales.where(condition)

df_sales_new = df_sales_changed.withColumn(
    "Quantity",
    when(condition, 123).otherwise(col("Quantity"))
)

# inserting a new row
new_row = Row(SalesOrderNumber="MOCK123", OrderDate="Friday, August 25, 2017", ProductKey=456, ResellerKey=123, EmployeeKey=123, SalesTerritoryKey=123, Quantity=456, UnitPrice="$10", Sales="$10", Cost="$10")
new_row_df = spark.createDataFrame([new_row])
df_sales_new = df_sales_new.union(new_row_df)

display(df_sales_new)


df_sales_new.write \
    .option("delimiter", "\t") \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save("Files/Bronze_AdventureWorks/Sales_new_29_04_2024.csv")

df_sales_new = spark.read.option("delimiter", "\t").format("csv").option("header", "True").load("Files/Bronze_AdventureWorks/Sales_new_29_04_2024.csv")
display(df_sales_new)

StatementMeta(, e77283dd-9b25-4e32-a7be-0f7b3e7168a9, 108, Finished, Available)

SynapseWidget(Synapse.DataFrame, c2fe3831-02d3-4fe3-9e55-4fd566dfd9e4)

SynapseWidget(Synapse.DataFrame, b00370bb-c2ed-4b94-889a-45a0109975a7)